# Libraries

In [1]:
import numpy as np
import pandas as pd

# Data import

source: https://www.kaggle.com/datasets/skamlo/wine-price-on-polish-market

In [ ]:
data = pd.read_csv("./data/wina.pl_raw.csv")
data.head(1)

# Print basic informations about columns

In [ ]:
data[sorted(data.columns)].info()

# Remove products which is not wines

There are a few product that are not a wine: `Chateau ISH Sparkling Rose w puszce (bezalkoholowe)`, `Chateau ISH Sparkling White w puszce (bezalkoholowe)`

In [70]:
data = data[~(data['name'] == 'Chateau ISH Sparkling Rose w puszce (bezalkoholowe)')]
data = data[~(data['name'] == 'Chateau ISH Sparkling White w puszce (bezalkoholowe)')]
data = data[data['ilość win'].isnull()]
data.reset_index(inplace=True, drop=True)

# Remove unnecessary columns

In [71]:
data.drop(['ilość win', 'wino koszerne', 'producent', 'wysyłka w ciągu', 'temperatura podawania2'], axis=1, inplace=True)

# Name

Related columns: `name`

Units in product name: `['ml', 'Ml', 'ML', 'l', 'L', 'litrów', 'NV']`

In [72]:
def is_number(text: str) -> bool:
    '''Check if text contain a number.'''
    for digit in [str(i) for i in range(10)]:
        if digit in text:
            return True
    return False

def is_unit(text: str) -> bool:
    if text in ['ml', 'Ml', 'ML', 'l', 'L', 'litrów', 'NV']:
        return True
    return False

def convert_name(name: str) -> str:
    words = []
    for word in name.split():
        if is_number(word):
            continue

        if is_unit(word):
            continue

        words.append(word)
    
    return " ".join(words)

data['name'] = data["name"].transform(convert_name)

# Price

Related columns: `price`

Watch out for \xa0 sign!

In [73]:
def convert_price(price:str) -> float:
    price = price.rstrip(" zł")
    price = price.replace(",", ".").replace(" ", "").replace(u"\xa0", "")
    return float(price)

data["price"] = data["price"].transform(convert_price)

# Region

Related columns: `region`, `region2`

In [74]:
def concat_region(row:pd.Series) -> pd.Series:
    if (not pd.isna(row["region"])) and (not pd.isna(row["region2"])):
        row["region"] += ", " + row["region2"]
    return row

data = data.transform(concat_region, axis=1)
data.drop(["region2"], axis=1, inplace=True)

# Vintage

Related columns: `rocznik`, `rocznik2`, `rocznik3`

In [ ]:
data[(data['rocznik2'].notnull()) | (data['rocznik3'].notnull())][["rocznik", "rocznik2", "rocznik3"]]

In [ ]:
data['rocznik'] = data['rocznik'].replace('NV', np.nan)

def choose_largest_vintage(row:pd.Series) -> pd.Series:
    vintages = []
    for vintage in row[["rocznik", "rocznik2", "rocznik3"]].to_list():
        try:
            vintage = int(vintage)
        except:
            continue
        vintages.append(vintage)
    
    row["rocznik"] = np.nan if vintages == [] else max(vintages)

    return row

data = data.transform(choose_largest_vintage, axis=1)
data.rename(columns={"rocznik": "vintage"}, inplace=True)
data.drop(['rocznik2', 'rocznik3'], axis=1, inplace=True)

# Volume

Related columns: `objętość`, `pojemność`

In [76]:
def fill_empty_volume(row:pd.Series) -> pd.Series:
    if not pd.isna(row["objętość"]) and pd.isna(row["pojemność"]):
        row["pojemność"] = row["objętość"]
    return row

data = data.transform(fill_empty_volume, axis=1)
data.drop(["objętość"], axis=1, inplace=True)

def convert_volume(volume:str) -> float:
    if not pd.isna(volume):
        volume, unit = volume.split()
        volume = volume.replace(",", ".")
        if unit == "ml":
            volume = float(volume) / 1000
        elif unit == "L":
            volume = float(volume)
    return volume

data['pojemność'] = data['pojemność'].transform(convert_volume)
data.rename(columns={"pojemność": "volume (liters)"}, inplace=True)

# Alcohol

Related columns: `alkohol zawartość`, `zawartość alkoholu2`, `zawartość alkoholu`

In [77]:
def fill_empty_alcohol(row:pd.Series) -> pd.Series:
    if not pd.isna(row["alkohol zawartość"]) and pd.isna(row["zawartość alkoholu"]):
        row["zawartość alkoholu"] = row["alkohol zawartość"]
    return row

data = data.transform(fill_empty_alcohol, axis=1)
data.drop(["alkohol zawartość", "zawartość alkoholu2"], axis=1, inplace=True)

def convert_alcohol(alcohol:str) -> float:
    if pd.isna(alcohol):
        return alcohol
    
    alcohol = alcohol.rstrip("%").rstrip(" ").replace(",", ".")
    return float(alcohol)

data["zawartość alkoholu"] = data["zawartość alkoholu"].transform(convert_alcohol)
data.rename(columns={"zawartość alkoholu": "alcohol (%)"}, inplace=True)

# Serving temperature

Related columns: `temperatura podawania`

In [78]:
def convert_temperature(temp:str) -> str:
    if pd.isnull(temp):
        return temp
    return temp.rstrip(" st.C")

data["temperatura podawania"] = data["temperatura podawania"].transform(convert_temperature)
data.rename(columns={"temperatura podawania": "serving temperature (C)"}, inplace=True)

# Color and kind

Related columns: `rodzaj wina`, `rodzaj wina2`

In [79]:
color = ['białe', 'czerwone', 'różowe', 'pomarańczowe']
kind = ['szampan', 'musujące', 'sherry', 'porto']

kolor = []
rodzaj = []

for rodzaj1, rodzaj2 in zip(data['rodzaj wina'], data['rodzaj wina2']):
    if rodzaj1 in color:
        kolor.append(rodzaj1)
    elif rodzaj2 in color:
        kolor.append(rodzaj2)
    else:
        kolor.append(np.nan)

    if rodzaj1 in kind:
        rodzaj.append(rodzaj1)
    elif rodzaj2 in kind:
        rodzaj.append(rodzaj2)
    else:
        rodzaj.append(np.nan)

data[['color', 'kind']] = pd.DataFrame({
    'color': kolor,
    'kind': rodzaj
})
data.drop(['rodzaj wina', 'rodzaj wina2'], axis=1, inplace=True)

# Medals

Related columns: `medale`, `medale2`, `medale3`

In [80]:
medals = []

for row in range(len(data)):
    medal = ''
    for value in data.loc[row, ['medale', 'medale2', 'medale3']]:
        if pd.isnull(value):
            break

        medal += value + ', '

    if medal == '':
        medals.append(np.nan)
    else:
        medals.append(medal[:-2])
    
data['medals'] = pd.DataFrame({'medals': medals})
data.drop(['medale', 'medale2', 'medale3'], axis=1, inplace=True)

# Wegan and Natural

Related columns: `bio`, `bio2`, `bio3`, `czy winnica bio`, `wino wegańskie`

In [81]:
wegan = []
natural = []

for bio, bio2, bio3, isBio, isWegan in zip(data['bio'], data['bio2'], data['bio3'], data['czy winnica bio'], data['wino wegańskie']):
    if bio == 'weganskie' or \
       bio2 == 'weganskie' or \
       bio3 == 'weganskie' or \
       isWegan == 'tak':
        wegan.append(True)
    else:
        wegan.append(False)

    if (bio in ['organiczne', 'naturalne']) or \
       (bio2 in ['organiczne', 'naturalne']) or \
       (bio3 in ['organiczne', 'naturalne']) or \
       isBio == 'tak – uprawa ograniczna/naturalna':
        natural.append(True)
    else:
        natural.append(False)

bio = pd.DataFrame({
    'wegan': wegan,
    'natural': natural
})

data = pd.concat([data, bio], axis=1)
data.drop(['bio', 'bio2', 'bio3', 'czy winnica bio', 'wino wegańskie'], axis=1, inplace=True)

# Punctation

Related columns: `guia penin`, `falstaff`, `james suckling`, `oceny prasy`, `wine spectator`

In [82]:
punctation = []

for i in range(len(data)):
    points = []

    if not pd.isnull(data.loc[i, 'guia penin']):
        points.append(int(data.loc[i, 'guia penin'][:2]))

    if not pd.isnull(data.loc[i, 'falstaff']):
        points.append(int(data.loc[i, 'falstaff'][-3:]))

    if not pd.isnull(data.loc[i, 'james suckling']):
        points.append(int(data.loc[i, 'james suckling'][:2]))

    if not pd.isnull(data.loc[i, 'oceny prasy']):
        points.append(int(data.loc[i, 'oceny prasy'][-3:]))

    if not pd.isnull(data.loc[i, 'wine spectator']):
        points.append(int(data.loc[i, 'wine spectator'][:2]))

    if len(points) > 0:
        punctation.append(np.mean(points))
    else:
        punctation.append(np.nan)

data['punctation'] = pd.Series(punctation).astype('float32')
data.drop(['guia penin', 'falstaff', 'james suckling', 'oceny prasy', 'wine spectator'], axis=1, inplace=True)

# Grapes

Related columns: `grona`, `grona[2-13]`

In [83]:
cols = ['grona'] + [f'grona{i}' for i in range(2, 14)]

grapes = []

for row in range(len(data)):
    grape = ''
    for value in data.loc[row, cols]:
        if pd.isnull(value):
            break

        grape += value + ', '

    if grape == '':
        grapes.append(np.nan)
    else:
        grapes.append(grape[:-2])
    
data['grapes'] = pd.DataFrame({'grapes': grapes})
data.drop(cols, axis=1, inplace=True)

# Appellation

Related columns: `apelacja`

Do something with label `- brak apelacji`

In [ ]:
data['apelacja'].replace('- brak apelacji', np.nan, inplace=True)
data.rename(columns={"apelacja": "appellation"}, inplace=True)

# Translate other columns

Replated columns: `kraj`, `winnica`, `smak`, `styl`

In [90]:
data.rename(columns={
    "kraj": "country",
    "winnica": "vineyard",
    "smak": "taste",
    "styl": "style"
}, inplace=True)

# Export clean dataset

In [ ]:
data.to_csv("./data/wina.pl_new_clean.csv", index=False)